In [ ]:
# Run this cell if you are using google colab
!pip install openai

In [9]:
from openai import OpenAI
# Uncomment the following line to get the API key from the user data
# if you run this in colab
# from google.colab import userdata
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# if you run this in local
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [10]:
from datetime import datetime

def get_current_timestamp():
  """
  Returns the current timestamp in the format "YYYY-MM-DD hh:mm:ss".
  """
  now = datetime.now()
  return now.strftime("%Y-%m-%d %H:%M:%S")



In [11]:
from datetime import datetime

def get_current_date():
  """
  Returns the current date in the format "YYYY-MM-DD".
  """
  now = datetime.now()
  return now.strftime("%Y-%m-%d")



In [12]:
# Uncomment the following line if you are using google colab
# from google.colab import drive

import json

def save_dict_to_drive(data, filename):
  """Saves a dictionary to a JSON file in Google Drive.

  Args:
    data: The dictionary to save.
    filename: The name of the file to save to (e.g., 'my_data.json').
  """
  # Uncomment the following lines if you are using google colab
  # drive.mount('/content/drive')
  # filepath = f'/content/drive/MyDrive/Colab/{filename}'

  # If you are not using google colab, you can save the file in the current directory
  filepath = f'./output/{filename}'
  with open(filepath, 'w') as f:
    json.dump(data, f, indent=4)
  print(f"Dictionary saved to: {filepath}")

def load_dict_from_drive(filename):
  """Loads a dictionary from a JSON file in Google Drive.

  Args:
    filename: The name of the file to load from (e.g., 'my_data.json').

  Returns:
    The loaded dictionary, or None if the file is not found.
  """
  # Uncomment the following lines if you are using google colab
  # drive.mount('/content/drive')
  # filepath = f'/content/drive/MyDrive/Colab/{filename}'

  # If you are not using google colab, you can load the file from the current directory
  filepath = f'./output/{filename}'
  try:
    with open(filepath, 'r') as f:
      data = json.load(f)
    print(f"Dictionary loaded from: {filepath}")
    return data
  except FileNotFoundError:
    print(f"File not found: {filepath}")
    return None


# Example usage:
# my_dict = {'key1': 'value1', 'key2': 'value2'}
# save_dict_to_drive(my_dict, 'my_dictionary.json')

# loaded_dict = load_dict_from_drive('my_dictionary.json')
# if loaded_dict:
#   print(loaded_dict)


In [7]:
data = load_dict_from_drive('task_patched.json')

Dictionary loaded from: ./output/task_patched.json


In [13]:
llm_methods = {
  "zero_shot": {
      "prompt": """
You are an AI language model specialized in determining causal relationships. When presented with a question about causality, analyze the information carefully and provide a clear, concise response. Your answer should:

Reasoning: Begin with a brief explanation of whether and how the cause leads to the effect.
Conclusion: End with "The answer is Yes." or "The answer is No." based on your analysis.
""" ,
      "file_name": "zs"
  },
  "cot": {
      "prompt": """
You are an AI language model tasked with answering questions about causation from the perspective of a typical person. When presented with a causation question, please follow these steps:

1. **Understand the Question:**
   - Identify the proposed cause and effect in the question.
   - Determine what is being asked and any common assumptions involved.

2. **Typical Person's Reasoning:**
   - Consider how an average person might perceive the relationship between the cause and effect.
   - Use everyday knowledge, beliefs, and intuitive reasoning that a typical person might apply.
   - Provide a brief explanation reflecting this common-sense reasoning.

3. **Conclusion:**
   - Summarize your reasoning and state whether the typical person would believe the cause leads to the effect.
   - End your response with "The answer is Yes." or "The answer is No." based on this perspective.

      """,
      "file_name": "cot"

  },
    "tot_step1": {
      "prompt": """
Expand the provided input text to make it more comprehensive and easier to answer questions related to the content.

# Steps

1. **Read the Input Text**: Carefully read the text to understand its main points and context.
2. **Identify Key Points**: Determine the main ideas, arguments, or pieces of information conveyed in the text.
3. **Expand Details**: Add relevant details, explanations, and examples that support and clarify the key points without deviating from the original context.
4. **Ensure Clarity**: Make sure the expanded text is clear and logically organized, making it simpler for someone to answer questions based on the information provided.
5. **Maintain Original Intent**: Ensure the expanded text remains true to the original meaning and intent of the input.

# Output Format

- A paragraph or series of paragraphs that are coherent, concise, and enriched with additional details to aid comprehension.
- Ensure the expanded text preserves the original message while enhancing clarity.

# Examples

**Input**: "Photosynthesis is the process by which plants use sunlight to produce food."

**Expanded Output**: "Photosynthesis is a vital biological process through which green plants, algae, and some bacteria transform light energy, typically from the sun, into chemical energy stored in glucose. This process primarily occurs in the chloroplasts of plant cells, where chlorophyll pigments capture sunlight. In addition to light, photosynthesis requires carbon dioxide, which plants absorb from the air, and water, taken up by roots from soil. As these molecules undergo a transformation facilitated by sunlight, oxygen is released as a by-product, contributing to the breathable atmosphere on Earth."

# Notes

- Be cautious not to alter the fundamental meaning of the input text while expanding.
- Consider the target audience's understanding level to determine the complexity of the additional details.
""" ,
      "file_name": "tot_step1"
  },
}


In [14]:
# Define the method to use
method = "tot_step1"
llm_params = llm_methods[method]

In [15]:
model_params = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": llm_params['prompt']},
        {
            "role": "user",
            "content": "example['input']"
        }
    ]
}

print(model_params['messages'][0])


{'role': 'system', 'content': '\nExpand the provided input text to make it more comprehensive and easier to answer questions related to the content.\n\n# Steps\n\n1. **Read the Input Text**: Carefully read the text to understand its main points and context.\n2. **Identify Key Points**: Determine the main ideas, arguments, or pieces of information conveyed in the text.\n3. **Expand Details**: Add relevant details, explanations, and examples that support and clarify the key points without deviating from the original context.\n4. **Ensure Clarity**: Make sure the expanded text is clear and logically organized, making it simpler for someone to answer questions based on the information provided.\n5. **Maintain Original Intent**: Ensure the expanded text remains true to the original meaning and intent of the input.\n\n# Output Format\n\n- A paragraph or series of paragraphs that are coherent, concise, and enriched with additional details to aid comprehension.\n- Ensure the expanded text pres

In [16]:
llm_output = {
    "description": "This file is the first step of the ToT method, it containts thoughts generated by the LLM for each question",
    "url": "https://github.com/google/BIG-bench/blob/main/bigbench/benchmark_tasks/causal_judgment/task.json",
    "model_params": model_params,
    "timretamp": get_current_timestamp(),
    "results" : []
}

In [26]:
import re
import copy
llm_output['results'] = copy.deepcopy(data['examples'])
client = OpenAI(
    api_key = OPENAI_API_KEY,
)

for idx, example in enumerate(llm_output['results']):
  print(f"Processing example {idx+1}/{len(llm_output['results'])}")
  text = example['input_text']

  for i in range(5):
    print(f"Creating thought {i+1}/5")
    completion = client.chat.completions.create(
        model=model_params['model'],
        messages=[
            model_params['messages'][0],
            {
                "role": "user",
                "content": text
            }
        ]
    )
    response = completion.choices[0].message.content
    if 'thoughts' not in example:
      example['thoughts'] = []
    example['thoughts'].append(response)
  #  print(response)
  #  example['thoughts'] = copy.deepcopy(response)
  if idx == 1:
    # Uncomment to stop the execution after the first 2 questions
    #break
    pass

import os.path
from os import path

date = get_current_date()
file_name = f"llm_output_{llm_params['file_name']}_{date}.json"

save_dict_to_drive(llm_output, file_name)

Processing example 1/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Creating thought 4/5
Creating thought 5/5
Processing example 2/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Creating thought 4/5
Creating thought 5/5
Processing example 3/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Creating thought 4/5
Creating thought 5/5
Processing example 4/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Creating thought 4/5
Creating thought 5/5
Processing example 5/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Creating thought 4/5
Creating thought 5/5
Processing example 6/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Creating thought 4/5
Creating thought 5/5
Processing example 7/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Creating thought 4/5
Creating thought 5/5
Processing example 8/190
Creating thought 1/5
Creating thought 2/5
Creating thought 3/5
Cr

In [26]:
print("Let's check if there are unparsed resonses from the LLM")
unparsed_responses = 0
for idx, task in enumerate(llm_output['results']):
  _cnt = 0
  for a in task['llm_answer']:
    _cnt += task['llm_answer'][a]
  if _cnt != 1:
    print(f"We could not determine the LLM for {_cnt} questions, check this tasks:")
    print(idx, task)
    unparsed_responses += 1

if unparsed_responses == 0:
  print("All responses were parsed correctly")
else:
  print(f"We could not determine the LLM for {unparsed_responses} questions")
  print("You need to manually check the answers for these questions")


Let's check if there are unparsed resonses from the LLM
All responses were parsed correctly


In [27]:
# Let's calculate the performance
score_cnt = 0

for idx, task in enumerate(llm_output['results']):
  score_cnt += task['score']

print(f"The performance of the model zero shot is {score_cnt/len(llm_output['results']) * 100:.2f}%")


The performance of the model zero shot is 64.74%


In [28]:
llm_output['performance'] = score_cnt/len(llm_output['results'])

In [23]:
import os.path
from os import path

date = get_current_date()
file_name = f"llm_output_{llm_params['file_name']}_{date}.json"

save_dict_to_drive(llm_output, file_name)

Dictionary saved to: ./output/llm_output_tot_step1_2024-10-15.json


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text Boxes with Arrows</title>
    <style>
        .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .box {
            width: 150px;
            height: 100px;
            border: 2px solid #333;
            border-radius: 10px;
            display: flex;
            justify-content: center;
            align-items: center;
            margin: 0 20px;
            font-family: Arial, sans-serif;
            font-size: 16px;
        }
        .arrow {
            width: 50px;
            height: 2px;
            background-color: #333;
            position: relative;
        }
        .arrow::after {
            content: '';
            position: absolute;
            right: 0;
            top: -4px;
            width: 0;
            height: 0;
            border-left: 10px solid #333;
            border-top: 5px solid transparent;
            border-bottom: 5px solid transparent;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="box">Text Box 1</div>
        <div class="arrow"></div>
        <div class="box">Text Box 2</div>
        <div class="arrow"></div>
        <div class="box">Text Box 3</div>
    </div>
</body>
</html>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text Boxes with Arrows</title>
    <style>
        .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .box {
            width: 150px;
            height: 100px;
            border: 2px solid #333;
            border-radius: 10px;
            display: flex;
            justify-content: center;
            align-items: center;
            margin: 0 20px;
            font-family: Arial, sans-serif;
            font-size: 16px;
        }
        .arrow {
            width: 50px;
            height: 2px;
            background-color: #333;
            position: relative;
        }
        .arrow::after {
            content: '';
            position: absolute;
            right: 0;
            top: -4px;
            width: 0;
            height: 0;
            border-left: 10px solid #333;
            border-top: 5px solid transparent;
            border-bottom: 5px solid transparent;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="box">Text Box 1</div>
        <div class="arrow"></div>
        <div class="box">Text Box 2</div>
        <div class="arrow"></div>
        <div class="box">Text Box 3</div>
    </div>
</body>
</html>